In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing necessary libraries
import seaborn as sns 
import matplotlib.pyplot as plt 

import tensorflow as tf  
from tensorflow.keras import layers
from tensorflow.keras import Model  
from tensorflow.keras.optimizers import RMSprop
from keras_preprocessing.image import ImageDataGenerator


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


#loading training and testing datasets
X_train_data = np.load("../input/mais202fall2021/train_images.npy")
y_train_data = pd.read_csv("../input/mais202fall2021/train_labels.csv")
X_test_data = np.load("../input/mais202fall2021/test_images.npy")



#Normalizing the data for better neural network training

X_train_data = X_train_data/255
X_test_data = X_test_data/255

#reshaping for CNN
X_train_data = X_train_data.reshape((50000, 28, 28, 1))
X_test_data = X_test_data.reshape((20000,28,28,1))

print(f"Shape of training data: {X_train_data.shape}")
print(f"Shape of testing data: {X_test_data.shape}")

y_train_data = pd.DataFrame([y_train_data["label"]])
y_train_data = y_train_data.T
print("Training labels: ")
y_train_data.head()


In [ ]:
#Viewing a sample image
plt.matshow(X_train_data[9]) 

**Creating the Neural Network Model**

In [ ]:
CNN_model = Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1),padding='valid',strides=(1,1)),
    #layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='valid',strides=(1,1)),
    layers.MaxPooling2D((2, 2),padding='valid',strides=None),
    
    layers.Dropout(0.5),
    
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu',padding='valid',strides=(1,1)),
    layers.MaxPooling2D((2, 2),padding='valid',strides=None),
    
    layers.Dropout(0.5),
    

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
    
])


In [ ]:
CNN_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
CNN_model.fit(X_train_data,y_train_data,epochs=30)

In [ ]:
y_test_results = CNN_model.predict(X_test_data)
y_test = []
for i in range(0,20000):
    y_test.append(np.argmax(y_test_results[i]))
     
ID = []
for i in range(0,20000):
    ID.append(i)

submission = pd.DataFrame(ID,columns=["ID"])
submission["label"] = y_test
submission

submission.to_csv('submission3.csv',index=False)
